# High Performance Computing in Python

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

n_observed = 100

cmap = cm.seismic
sb.set(context='notebook', rc={"figure.figsize": (18, 5)})

hr = pd.read_csv('data/hr2.txt', names=['hr'], dtype=np.float32)
observed = hr.sample(n_observed).sort_index()


x, y = hr.index.values, hr.values[:, 0]
obs_x, obs_y = observed.index.values, observed.values[:, 0]


plt.plot(obs_x, obs_y)
plt.plot(obs_x, obs_y, 'ro')

In [ ]:
def delta(i,j):
    return i==j

def covariance(x, y, kernel=delta):
    n,m = len(x), len(y)
    C = np.empty((n,m))
    for i in range(n):
        for j in range(m):
            C[i,j] = kernel(x[i], y[j])
    return C

plt.imshow(covariance(obs_x, obs_x))

In [ ]:
covariance(obs_x, obs_x)

In [ ]:
plt.plot(covariance(x[[900,1100]], x, kernel=delta).T)

In [ ]:
def brownian(i, j):
    return min(i,j)

plt.imshow(covariance(obs_x, obs_x, kernel=brownian), cmap=cmap)

In [ ]:
plt.plot(covariance(x[[900,1100]], x, kernel=brownian).T)

In [ ]:
theta = np.float32(0.001)

def laplace(i, j):|
    return np.exp(-theta * np.abs(i - j))

plt.matshow(covariance(obs_x, obs_x, kernel=laplace), cmap=cmap)

In [ ]:
plt.plot(covariance(x[[900,1100]], x, kernel=laplace).T)

In [ ]:
theta2 = np.float32(0.001)**2

def gaussian(i, j):
    return np.exp(-theta2 * np.abs(i - j)**2)

plt.matshow(covariance(obs_x, obs_x, kernel=gaussian), cmap=cmap)

In [ ]:
plt.plot(covariance(x[[900,1100]], x, kernel=gaussian).T)

In [ ]:
def predict(new_x, x, y, kernel=delta):
    c_new_x = covariance(new_x, x, kernel=kernel)
    c_x = covariance(x, x, kernel=kernel)
    solve = np.linalg.solve(c_x, y)
    r = c_new_x.dot(solve)
    return r

In [ ]:
def plot_predict(kernel=delta):
    new_x = hr.index.values
    new_y = predict(new_x, obs_x, obs_y, kernel=kernel)

    plt.plot(new_x, new_y, label='Prediction')
    plt.plot(obs_x, obs_y, 'ro', label='Observation')
    plt.plot(x, y, label='Real')
    plt.legend()
    return new_x, new_y

In [ ]:
new_x, new_y = plot_predict(delta)

In [ ]:
new_x, new_y = plot_predict(brownian)

In [ ]:
theta = 0.01
new_x, new_y = plot_predict(laplace)

In [ ]:
theta2 = 0.01**2
new_x, new_y = plot_predict(gaussian)
plt.xlim([100, 1800])
plt.ylim([75, 110])

In [ ]:
theta2 = 0.01**2
new_x, new_y = plot_predict(lambda x,y: 0.001*delta(x,y) + gaussian(x,y))

In [ ]:
plt.matshow(covariance(obs_x, obs_x, kernel=lambda x,y: 0.1*delta(x,y) + gaussian(x,y)), cmap=cmap)

In [ ]:
plt.matshow(covariance(obs_x, obs_x, kernel=lambda x,y: gaussian(x,y)), cmap=cmap)

In [ ]:
plt.plot(covariance(x[[900,1100]], x, kernel=lambda x,y: 0.1*delta(x,y) + gaussian(x,y)).T)

In [ ]:
def plot_my_prediction(noisy=10.0):
    plot_predict(lambda x,y: delta(x,y) + noisy*gaussian(x,y))
    plt.show()
    
plot_my_prediction()

In [ ]:
from ipywidgets import interact_manual, FloatSlider

intervals = dict()
intervals['noisy'] = FloatSlider(min=0.0, max=20.0, value=10, step=1e-2)

interact_manual(plot_my_prediction, **intervals)

# Profiling

In [ ]:
%time plot_my_prediction()

In [ ]:
%timeit plot_my_prediction()

In [ ]:
#https://pypi.org/project/memory_profiler/
%reload_ext memory_profiler

In [ ]:
%memit plot_my_prediction()

In [ ]:
%memit new = np.random.rand(int(2e8))

In [ ]:
%prun plot_my_prediction()

In [ ]:
!pip install line-profiler

In [ ]:
#https://pypi.org/project/line_profiler/
%reload_ext line_profiler

In [ ]:
%lprun -f plot_my_prediction plot_my_prediction()

In [ ]:
%lprun -f plot_predict plot_my_prediction()

In [ ]:
%lprun -f predict plot_my_prediction()

In [ ]:
%lprun -f covariance plot_my_prediction()

In [ ]:
%lprun -f delta -f gaussian plot_my_prediction()

In [ ]:
plot_my_prediction()

# Multiprocessing

In [ ]:
def parallel_function(index):
    for i in range(30000000):
        i**2
    print(index)
    return index

In [ ]:
parallel_function(0)

In [ ]:
import multiprocessing as mp

processes = 8


p = mp.Pool(processes)
p

In [ ]:
p.map(parallel_function, list(range(processes)))

In [ ]:
kernel = lambda x,y: delta(x,y) + theta2*gaussian(x,y)

covariance(x, x, kernel)

In [ ]:
def partial_covariance(x, y, kernel=delta, n0=0, nf=None):
    if nf is None:
        nf = len(x)
    m = len(y)
    C = np.empty((nf-n0,m))
    for i in range(n0, nf):
        for j in range(m):
            C[i - n0,j] = kernel(x[i], y[j])
    return C

In [ ]:
%%time
partial_covariance(x, x, kernel)

In [ ]:
%%time
pcov1 = partial_covariance(x, x, kernel, n0=0, nf=len(x)//2)
pcov2 = partial_covariance(x, x, kernel, n0=len(x)//2, nf=len(x))
np.concatenate([pcov1, pcov2])

In [ ]:
processes = 8

def parallel_covariance(index):
    step = len(x) // processes
    n0 = step * index
    nf = step * (index + 1)
    if index + 1 == processes:
        nf = len(x)
    m = len(x)
    C = np.empty((nf - n0, m))
    for i in range(n0, nf):
        for j in range(m):
            C[i - n0, j] = kernel(x[i], x[j])
    return C

In [ ]:
p = mp.Pool(processes)

In [ ]:
%%time
np.concatenate(p.map(parallel_covariance, list(range(processes))))

# Numba

In [ ]:
import numpy as np

n = int(1e4)
r = np.random.randn(n*n).reshape((n,n))

In [ ]:
def sum2d(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

In [ ]:
sum2d(r)

In [ ]:
from numba import jit

@jit
def sum2d_jit(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

In [ ]:
sum2d_jit(r)

In [ ]:
theta2 = np.float32(0.001)**2
kernel =  lambda x,y: delta(x,y) + theta2*gaussian(x,y)

%time covariance(x, obs_x, kernel)

In [ ]:
@jit
def jit_gaussian(i, j):
    return np.exp(-theta2 * np.abs(i - j)**2)

@jit
def jit_delta(i,j):
    return i==j
   
@jit
def jit_covariance(x, y):
    n,m = len(x), len(y)
    C = np.empty((n,m))
    for i in range(n):
        for j in range(m):
            C[i,j] = jit_delta(x[i], y[j]) + theta2*jit_gaussian(x[i], y[j])
    return C

In [ ]:
%time jit_covariance(x, obs_x)

In [ ]:
theta2 = 0.01**2
covariance(x, obs_x, kernel)

In [ ]:
jit_covariance(x, obs_x)

In [ ]:
@jit
def jit_gaussian(i, j, theta2):
    return np.exp(-theta2 * np.abs(i - j)**2)
   
@jit
def jit_covariance(x, y, theta2, noisy=10):
    n,m = len(x), len(y)
    C = np.empty((n,m))
    for i in range(n):
        for j in range(m):
            C[i,j] = jit_delta(x[i], y[j]) + noisy*jit_gaussian(x[i], y[j], theta2)
    return C

In [ ]:
jit_covariance(x, obs_x, theta2)

In [ ]:
%%time
predict(x, obs_x, obs_y, kernel)

In [ ]:
@jit
def jit_predict(new_x, x, y, noisy=10):
    return jit_covariance(new_x, x, theta2, noisy).dot(np.linalg.solve(jit_covariance(x, x, theta2, noisy), y))

In [ ]:
%%time
jit_predict(x, obs_x, obs_y)

In [ ]:
def jit_my_prediction(noisy=10.0):
    new_x = hr.index.values
    new_y = jit_predict(new_x, obs_x, obs_y, noisy)

    plt.plot(new_x, new_y, label='Prediction')
    plt.plot(obs_x, obs_y, 'ro', label='Observation')
    plt.plot(x, y, label='Real')
    plt.legend()
    plt.show()

In [ ]:
%time jit_my_prediction()

In [ ]:
from ipywidgets import interact_manual, FloatSlider

intervals = dict()
intervals['noisy'] = FloatSlider(min=0.0, max=20.0, value=10, step=1e-2)

interact_manual(plot_my_prediction, **intervals)

In [ ]:
interact_manual(jit_my_prediction, **intervals)

In [ ]:
from ipywidgets import interact

interact(jit_my_prediction, **intervals)

In [ ]:
%prun jit_my_prediction()

In [ ]:
%timeit gaussian(x[:, None], x)

In [ ]:
from numba import vectorize, float32
from math import exp

theta2 = 0.001**2

@vectorize([float32(float32, float32)])
def v_gaussian(i, j):
    return exp(-theta2 * abs(i - j)**2)


In [ ]:
%timeit v_gaussian(x[:,None].astype(np.float32), x.astype(np.float32))

In [ ]:
x_32 = x[:,None].astype(np.float32)
x32 = x.astype(np.float32)

In [ ]:
%timeit v_gaussian(x_32, x32)

In [ ]:
@vectorize([float32(float32, float32, float32, float32)])
def vec_covariance(i, j, theta2, noisy):
    return (i==j) + noisy*exp(-theta2 * abs(i - j)**2)

@jit
def vec_predict(new_x, x, y, noisy=10):
    return vec_covariance(new_x[:,None], x, theta2, noisy).dot(np.linalg.solve(vec_covariance(x[:,None], x, theta2, noisy), y))

In [ ]:
%timeit predict(x, obs_x, obs_y, kernel)

In [ ]:
%timeit jit_predict(x, obs_x, obs_y)

In [ ]:
obs_x32 = obs_x.astype(np.float32)
obs_y32 = obs_y.astype(np.float32)

In [ ]:
%timeit vec_predict(x_32, obs_x32 , obs_y32)